In [89]:
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pixel as pix
import utilities as util
import tile

In [90]:
os.environ["CUDA_VISIBLE_DEVICES"]="1";

In [91]:
landsat, sentinel, dem, label = util.load_data()
tile_size = 64
num_classes = util.get_class_count()
pixel_gen = pix.pixel_gen(landsat, sentinel, dem, label, tile_size, num_classes)
tile_gen = tile.tile_gen(landsat, sentinel, dem, label, tile_size, num_classes)

In [107]:
pixels = pixel_gen.gen_pixels(60000, index=4, balanced=True)

pixels generated 36307


In [108]:
train_px, test_px, val_px = pixel_gen.train_val_test_split(pixels, 0.7, 0.7)
pixel_gen.print_balance()

train:25414 val:7625 test:5556
Water:3750.0
Snow/Ice:0.0
Open Space Developed:3750.0
Low Intensity Developed:0.0
Medium Intensity Developed:3750.0
High Intensity Developed:0.0
Barren Land:3750.0
Deciduous Forest:144.0
Evergreen Forest:3750.0
Mixed Forest:420.0
Scrub/Shrub:3750.0
Grassland / Herbaceous:1993.0
Pasture/Hay:0.0
Cultivated Land:3750.0
Woody Wetland:3750.0
Emergent Herbaceous Wetlands:3750.0


In [109]:
import resnet as rs
from keras.optimizers import Adam

In [110]:
epochs = 10
batch_size = 25
input_shape = tile_gen.get_tile_shape(reshape=True)
print(input_shape)

(10, 64, 64)


In [ ]:
model = rs.ResnetBuilder.build_resnet_18(input_shape, 16)
opt = Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(generator=tile_gen.tile_generator(train_px, batch_size), 
                    steps_per_epoch=len(train_px) // batch_size, epochs=epochs, verbose=1,
                    validation_data=tile_gen.tile_generator(val_px, batch_size),
                    validation_steps=len(val_px) // batch_size)

Epoch 1/10
 874/1016 [========================>.....] - ETA: 15s - loss: 1.7440 - acc: 0.6130